In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from functions import mean_gen, varx_gen, vary_gen, spherical_data, test_on_integers, learn_parameters, likelihood_ratio, compare_learning
from tensorflow.python.framework.ops import disable_eager_execution

2022-08-23 12:53:16.016837: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-23 12:53:16.016871: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Random Seed
rand_n = 12345

# Single Point on Sphere
n = 10000
angles = np.array([2, 2])
thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)


# Likelihood Ratio of Data
l_out = likelihood_ratio(x_train)

2022-08-23 12:53:19.354085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-23 12:53:19.354121: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-23 12:53:19.354144: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-08-23 12:53:19.354598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [57]:
# KERNEL INITIALIZER

s = tf.convert_to_tensor([1.]).shape
res = tf.random.uniform(s, maxval=2*np.pi, seed=rand_n), tf.random.uniform(s, maxval=np.pi, seed=rand_n)

def my_init(shape, dtype=None):
    return tf.transpose(tf.convert_to_tensor(res))

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.3369558], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.43495548], dtype=float32)>)


In [58]:
# LIKELIHOOD MODEL

loss_fn0 = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold0 = tf.keras.Input(shape=(1,))
simple_linear0 = Dense(2, use_bias = False, kernel_initializer=my_init)(inputs_hold0)
model_angles0 = Model(inputs = inputs_hold0, outputs = simple_linear0)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs0 = tf.keras.Input(shape=(2,))
inputs0 = tf.keras.layers.concatenate([raw_inputs0, model_angles0(tf.ones_like(raw_inputs0)[:,0:1])])
output0 = likelihood_ratio(inputs0)

model_paramfinder = Model(inputs = raw_inputs0, outputs = output0)
model_paramfinder.compile(loss=loss_fn0, optimizer='Adam')
# model_paramfinder.summary()

In [59]:
# INTERPOLATED MODEL

model_interpolate = tf.keras.models.load_model('3dmodels/discrete_model_mth50_mph50')

for l in model_interpolate.layers:
    l.trainable=False
    
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Building model_angles which is used to train (theta, phi)
inputs_hold = tf.keras.Input(shape=(1,))
simple_linear = Dense(2, use_bias = False, kernel_initializer=my_init)(inputs_hold)
model_angles = Model(inputs = inputs_hold, outputs = simple_linear)

# Building model_parmafinder, inputs, which takes the (x, y) and finds the best (theta, phi)
raw_inputs = tf.keras.Input(shape=(2,))
inputs = tf.keras.layers.concatenate([raw_inputs, model_angles(tf.ones_like(raw_inputs)[:,0:1])])
output = model_interpolate(inputs)

model_interpolate_paramfinder = Model(inputs = raw_inputs, outputs = output)
model_interpolate_paramfinder.compile(loss=loss_fn, optimizer='Adam')
# model_interpolate_paramfinder.summary()



x_inputs = x_train[:, 0:2]

In [60]:
epochs, iterations, batch_size = 1, 50, 300

# # Likelihood Stats
# weights0 = learn_parameters(model_paramfinder, x_train, y_train, iterations, epochs, batch_size)
# angle_errors0 = (angles - weights0) / angles
# average_error0 = np.sum(np.abs(angle_errors0)) / 2


# Comparing
weights1, weights2 = compare_learning(model_paramfinder, model_interpolate_paramfinder, angles,
                                      x_train, y_train, iterations, epochs, batch_size)
# angle_errors = (angles - weights) / angles
# average_error = np.sum(np.abs(angle_errors)) / 2

# print('Average likelihood error %:', average_error0 * 100)
# print('Average interpolated error %:', average_error * 100)
# print('')

Initialized at  tf.Tensor([3.3369558  0.43495548], shape=(2,), dtype=float32) tf.Tensor([3.3369558  0.43495548], shape=(2,), dtype=float32)
54/54 [==============================] - 1s 4ms/step - loss: 0.6829
0 Fitted result 1:  tf.Tensor([3.2861183  0.48594213], shape=(2,), dtype=float32)
0 Fitted result 2:  tf.Tensor([3.2914474 0.4844382], shape=(2,), dtype=float32)
After  0 iterations, LE %:  70.004403591156  IE %:  70.17523050308228
54/54 [==============================] - 0s 4ms/step - loss: 0.6820
1 Fitted result 1:  tf.Tensor([3.2328894 0.5383812], shape=(2,), dtype=float32)
1 Fitted result 2:  tf.Tensor([3.2415318  0.53814834], shape=(2,), dtype=float32)
After  1 iterations, LE %:  67.36270189285278  IE %:  67.58458614349365
54/54 [==============================] - 0s 4ms/step - loss: 0.6805
2 Fitted result 1:  tf.Tensor([3.1771977 0.5902036], shape=(2,), dtype=float32)
2 Fitted result 2:  tf.Tensor([3.1868691 0.5954765], shape=(2,), dtype=float32)
After  2 iterations, LE %:  64

54/54 [==============================] - 0s 4ms/step - loss: 0.6303
22 Fitted result 1:  tf.Tensor([2.253146  1.5389323], shape=(2,), dtype=float32)
22 Fitted result 2:  tf.Tensor([2.0995288 1.653136 ], shape=(2,), dtype=float32)
After  22 iterations, LE %:  17.855340242385864  IE %:  11.15981936454773
54/54 [==============================] - 0s 4ms/step - loss: 0.6296
23 Fitted result 1:  tf.Tensor([2.2326257 1.5748563], shape=(2,), dtype=float32)
23 Fitted result 2:  tf.Tensor([2.06621   1.6822424], shape=(2,), dtype=float32)
After  23 iterations, LE %:  16.444236040115356  IE %:  9.599190950393677
54/54 [==============================] - 0s 4ms/step - loss: 0.6289
24 Fitted result 1:  tf.Tensor([2.2172089 1.606917 ], shape=(2,), dtype=float32)
24 Fitted result 2:  tf.Tensor([2.0364618 1.7093321], shape=(2,), dtype=float32)
After  24 iterations, LE %:  15.257295966148376  IE %:  8.178243041038513
54/54 [==============================] - 0s 4ms/step - loss: 0.6283
25 Fitted result 1: 

54/54 [==============================] - 0s 3ms/step - loss: 0.6255
44 Fitted result 1:  tf.Tensor([2.0361776 2.0033855], shape=(2,), dtype=float32)
44 Fitted result 2:  tf.Tensor([1.7359282 1.9459317], shape=(2,), dtype=float32)
After  44 iterations, LE %:  0.989079475402832  IE %:  7.953503727912903
54/54 [==============================] - 0s 4ms/step - loss: 0.6258
45 Fitted result 1:  tf.Tensor([2.0308297 2.0074623], shape=(2,), dtype=float32)
45 Fitted result 2:  tf.Tensor([1.7364128 1.9471407], shape=(2,), dtype=float32)
After  45 iterations, LE %:  0.9572982788085938  IE %:  7.911163568496704
54/54 [==============================] - 0s 4ms/step - loss: 0.6253
46 Fitted result 1:  tf.Tensor([2.0248582 2.0157583], shape=(2,), dtype=float32)
46 Fitted result 2:  tf.Tensor([1.7319341 1.952072 ], shape=(2,), dtype=float32)
After  46 iterations, LE %:  1.0154128074645996  IE %:  7.899847626686096
54/54 [==============================] - 0s 3ms/step - loss: 0.6261
47 Fitted result 1:  

In [14]:
weights1

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-1.0987433, -1.8309702], dtype=float32)>

In [33]:
# MANUAL TESTING

# xr = model_interpolate(x_train).numpy().transpose()[0]
# y = model_interpolate(x_train)
# loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
# print(loss)

# x_train_mod = np.copy(x_train)
# x_train_mod[:, 2] = x_train[:, 2]
# x_train_mod[:, 3] = x_train[:, 3] - .1
# xr_mod = model_interpolate(x_train_mod).numpy().transpose()[0]
# y_mod = model_interpolate(x_train_mod)
# loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
# print(loss_mod)

# xr = likelihood_ratio(x_train).numpy().transpose()
# y = likelihood_ratio(x_train)
# loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
# print(loss)

# x_train_mod = np.copy(x_train)
# x_train_mod[:, 2] = x_train[:, 2] - .05
# x_train_mod[:, 3] = x_train[:, 3] - .05
# xr_mod = likelihood_ratio(x_train_mod).numpy().transpose()
# y_mod = likelihood_ratio(x_train_mod)
# loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
# print(loss_mod)

0.6019409317687167
